# **Tabela Estoque**

Criação das colunas:

- Qt. Disponível = Qt. Gerencial - Qt. Bloqueada - Qt. Reservada
- Qt. Depósito = Qt. Gerencial - Qt. Frente Loja

Tratamento para que produtos que não tenham quantidade bloqueada entrem com codbloqueio e motivo de bloqueio NULOS

In [3]:
CREATE OR ALTER PROCEDURE SP_CARREGA_ESTOQUE
    @CODFILIAL          NVARCHAR(2),
    @CODPROD            INT,
    @QTCONTABIL         NUMERIC(22,8),
    @QTGERENCIAL        NUMERIC(22,8),
    @QTBLOQUEADA        NUMERIC(20,6),
    @QTPENDENTE         NUMERIC(16,3),
    @QTRESERVADA        NUMERIC(22,6),
    @QTAVARIADA         NUMERIC(20,6),
    @QTFRENTELOJA       NUMERIC(22,6),
    @VALORULTENTRADA    NUMERIC(18,6),
    @CUSTOREPOSICAO     NUMERIC(18,6),
    @CUSTOFINANCEIRO    NUMERIC(18,6),
    @CUSTOCONTABIL      NUMERIC(18,6),
    @CODBLOQUEIO        INT,
    @MOTIVOBLOQUEIO     NVARCHAR(30)
AS
BEGIN
    DECLARE @DTUPDATE DATETIME = DATEADD(HOUR, -3, GETUTCDATE())
    DECLARE @QTDISPONIVEL NUMERIC(22,8) = @QTGERENCIAL - @QTBLOQUEADA - @QTRESERVADA
    DECLARE @QTDEPOSITO NUMERIC(22,6) = @QTGERENCIAL - @QTFRENTELOJA
    
    --TRATANDO OS RETORNOS NULOS
    SET @MOTIVOBLOQUEIO = 
        CASE WHEN @MOTIVOBLOQUEIO = 'NULL' THEN NULL 
             WHEN @QTBLOQUEADA = 0 THEN NULL
        ELSE TRIM(UPPER(@MOTIVOBLOQUEIO)) END
    
    SET @CODBLOQUEIO = CASE WHEN @QTBLOQUEADA = 0 THEN NULL ELSE @CODBLOQUEIO END
    
    --REALIZANDO O MERGE
    IF EXISTS (SELECT 1 FROM ESTOQUE WHERE CODFILIAL = @CODFILIAL AND CODPROD = @CODPROD)
    BEGIN
        UPDATE ESTOQUE
            SET QTCONTABIL = @QTCONTABIL,
                QTGERENCIAL = @QTGERENCIAL,
                QTBLOQUEADA = @QTBLOQUEADA,
                QTPENDENTE = @QTPENDENTE,
                QTRESERVADA = @QTRESERVADA,
                QTAVARIADA = @QTAVARIADA,
                QTDISPONIVEL = @QTDISPONIVEL,
                QTFRENTELOJA = @QTFRENTELOJA,
                QTDEPOSITO = @QTDEPOSITO,
                VALORULTENTRADA = @VALORULTENTRADA,
                CUSTOREPOSICAO = @CUSTOREPOSICAO,
                CUSTOFINANCEIRO = @CUSTOFINANCEIRO,
                CUSTOCONTABIL = @CUSTOCONTABIL,
                CODBLOQUEIO = @CODBLOQUEIO,
                MOTIVOBLOQUEIO = @MOTIVOBLOQUEIO,
                DTUPDATE = @DTUPDATE
          WHERE CODFILIAL = @CODFILIAL AND CODPROD = @CODPROD
    END
    ELSE 
    BEGIN
        INSERT INTO ESTOQUE (
            CODFILIAL,
            CODPROD,
            QTCONTABIL,
            QTGERENCIAL,
            QTBLOQUEADA,
            QTPENDENTE,
            QTRESERVADA,
            QTAVARIADA,
            QTDISPONIVEL,
            QTFRENTELOJA,
            QTDEPOSITO,
            VALORULTENTRADA,
            CUSTOREPOSICAO,
            CUSTOFINANCEIRO,
            CUSTOCONTABIL,
            CODBLOQUEIO,
            MOTIVOBLOQUEIO,
            DTUPDATE)
    VALUES (
            @CODFILIAL,
            @CODPROD,
            @QTCONTABIL,
            @QTGERENCIAL,
            @QTBLOQUEADA,
            @QTPENDENTE,
            @QTRESERVADA,
            @QTAVARIADA,
            @QTDISPONIVEL,
            @QTFRENTELOJA,
            @QTDEPOSITO,
            @VALORULTENTRADA,
            @CUSTOREPOSICAO,
            @CUSTOFINANCEIRO,
            @CUSTOCONTABIL,
            @CODBLOQUEIO,
            @MOTIVOBLOQUEIO,
            @DTUPDATE)                          
    END
END

Commands completed successfully.

Total execution time: 00:00:00.016

In [ ]:
CREATE OR ALTER PROCEDURE SP_RETORNA_ESTOQUE
    @CODFILIAL NVARCHAR(2),
    @CODPROD   INT
AS
BEGIN    
    IF EXISTS (SELECT 1 FROM ESTOQUE WHERE CODFILIAL = @CODFILIAL AND CODPROD = @CODPROD)
	SELECT FORMAT(DTUPDATE, 'yyyy-MM-dd HH:mm:ss') DTUPDATE FROM ESTOQUE WHERE CODFILIAL = @CODFILIAL AND CODPROD = @CODPROD
END

In [5]:
SELECT * FROM ESTOQUE